# Towards a full version of french wino (split schemas)

Le but c'est de diviser les schémas en deux catégories : pmi-able et non.

In [5]:
import os,sys,inspect
import pandas as pd
import numpy as np
import importlib
import sys
from utils_xo import *
importlib.reload(sys.modules["utils_xo"])
from utils_xo import *
# set data directory
data_dir = r"../../../data/french/"

In [118]:
version_no_id = xo_read_json(data_dir+"pmi_frwac_alt_corrected.json")

In [119]:
## add english id from the xml file
import xml.etree.ElementTree as ET
import re
tree = ET.parse(data_dir + 'winograd_fr_v2.xml')
root = tree.getroot()

In [120]:
#! create the french-english correspondence
ids_fr_en = {}
for elem in root.findall("./"):
    french_id = elem.attrib['id']
    english_id = elem.attrib['engn']
    ids_fr_en[french_id] = english_id
    # for subelem in elem:
        # print(subelem.attrib['id'])

In [121]:
#! check if each french is in the xml file
#? get all the french schemas in the json
schemas_json = []
for wino in version_no_id:
    schemas_json.append(wino['schema'])
#? check if each schema in json is in xml
for fr_schema in schemas_json:
    if fr_schema not in ids_fr_en:
        print(fr_schema)

# 81 28s 28a all corrected by hand

28
28


In [122]:
#! add the id to json dict
for schema in version_no_id:
    if schema["schema"] != 28:
        schema["id"] = str(ids_fr_en[schema["schema"]])

In [43]:
#! drop schema key
# for schema in version_no_id:
#     schema.pop("schema",None)

In [123]:
version_id_no000 = []
for di in version_no_id:
    if di["id"] != "000":
        version_id_no000.append(di)
for di in version_id_no000:
    di["id"] = int(di["id"])
versions_id_ordered = sorted(version_id_no000,key=lambda x : x["id"])

In [124]:
for dt in versions_id_ordered:
    dt["french_schema_no"] = dt["schema"]
    dt.pop("schema",None)

In [126]:
versions_id_ordered
#! make id the first key
version_id_ordered = [{"id": di["id"], **di} for di in versions_id_ordered]

In [135]:
version_id_ordered[:3]

[{'id': 1,
  'pmi_able': 1,
  'nb_npropre': 0,
  'r0': 'conseillers municipaux',
  'r1': 'manifestants',
  'special': 'craignait',
  'alternate': 'pronait',
  'question': 'Qui prônait la violence ?',
  'french_schema_no': '58'},
 {'id': 3,
  'pmi_able': 1,
  'nb_npropre': 0,
  'r0': 'coupe',
  'r1': 'valise',
  'special': 'grand',
  'alternate': 'petit',
  'question': "Qu'est-ce qui est trop grand/petit ?",
  'french_schema_no': '1'},
 {'id': 5,
  'pmi_able': 1,
  'nb_npropre': 2,
  'r0': 'Claire',
  'r1': 'Suzanne',
  'special': 'demandé',
  'alternate': 'consacré',
  'question': 'Qui a demandé du temps ?',
  'french_schema_no': '59'}]

In [133]:
# xo_write_json(data_dir+"pmi_frwac_withid.json",version_id_ordered)
with open(data_dir+"pmi_frwac_withid.json", 'w' ,encoding='utf-8') as outfile:
    json.dump(version_id_ordered, outfile, ensure_ascii=False)